In [2]:
import numpy as np
import pandas as pd
import itertools as it
from collections import Counter
import calendar as clnd

In [3]:
data = pd.read_csv('C:\Projects_0\input_data\movie_bd_v5.xls')

In [4]:
data['profit'] = data.revenue - data.budget

In [5]:
data.describe()

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
 14  profit                1889 non-null   int64  
dtypes: float64(1), int64(

### Попробуем понять, можем ли мы заранее предугадать, какой фильм будет прибыльным
#### попытка 1 
- у нас еть рейтинг! проверим, есть ли связь между рейтингом и прибыльностью хотя, стоит проверять не только прибыльность, но и кассовые сборы

In [7]:
data_profit = data[data.profit > 0]              # датасет с прибыльными фильмами
data_loose = data[data.profit <= 0]              # датасет с убыточными фильмами
len(data_profit), len(data_loose)

(1478, 411)

In [8]:
data_profit.vote_average.describe()

count    1478.000000
mean        6.240122
std         0.738106
min         3.400000
25%         5.800000
50%         6.200000
75%         6.700000
max         8.100000
Name: vote_average, dtype: float64

In [9]:
data_loose.vote_average.describe()

count    411.000000
mean       5.783455
std        0.753022
min        3.300000
25%        5.300000
50%        5.800000
75%        6.300000
max        7.800000
Name: vote_average, dtype: float64

Несмотря на ожидания, средний рейтинг отличается крайне мало. Не показательны также и минимумы и максимумы.

### попытка 2
- проверим, есть ли связь между рейтингом и кассовыми сборами

In [10]:
data_revenue = data[data.revenue > data.revenue.mean()]     # датасет с касоовыми сборами выше среднего
data_under = data[data.revenue <= data.revenue.mean()]      # датасет с кассовыми сборами ниже среднего
len(data_revenue), len(data_under)

(560, 1329)

In [11]:
data_revenue.vote_average.describe()

count    560.000000
mean       6.396429
std        0.706870
min        4.400000
25%        5.900000
50%        6.400000
75%        6.900000
max        8.100000
Name: vote_average, dtype: float64

In [12]:
data_under.vote_average.describe()

count    1329.000000
mean        6.033032
std         0.762881
min         3.300000
25%         5.500000
50%         6.000000
75%         6.500000
max         8.000000
Name: vote_average, dtype: float64

### попытка 3
- углубимся в жанры и проанализируем кассовые сборы внутри групп

In [22]:
# переделаем строку с жанрами в список, клонируем фильмы для каждого жанра отдельно
data_genres = data.copy()
data_genres.genres = data_genres.genres.str.split('|')
data_genres = data_genres.explode('genres')

# сгруппируем по жанрам и посчитаем статитику кассовых сборов
rev = data_genres.groupby('genres').revenue.agg(['median', 'mean','max','min'])
rev.astype('int64').sort_values('median', ascending=False).nlargest(4, 'median')

,median,mean,max,min
genres,,,,
Animation,248384621,317054839,1274219009,5409517
Fantasy,180777735,321273643,2781505847,2333684
Adventure,169956806,305274016,2781505847,2033165
Family,155282309,261829828,1327817822,3878993


Вот это похоже на ответ! Медиана кассовых сборов в 250 млн. долларов - это беспроигрышный вариант - это Анимация!

Вообще показательно, что список топа выглядит: Анимация, Фэнтези, Приключения, Семейный. Это все жанры, куда идут семьями, то есть аудитория точно больше чем Экшн, Триллер или Драма!!!

Наверняка не только жанр играет роль в больших кассовых сборах. Хотя уже можно предположить, что большинство актеров или режиссеров так или иначе будут пересекатся с жанрами Анимации, Фэнтези и Путешествий. Но, попробуем этот  вопрос прояснить

In [53]:
# переделаем строку с режиссерами в список, клонируем фильмы для каждого режиссера отдельно
data_dir = data.copy()
data_dir.director = data_dir.director.str.split('|')
data_dir = data_dir.explode('director')

# сгруппируем по режиссерам и посчитаем статитику кассовых сборов
dir = data_dir.groupby('director').revenue.agg(['median', 'mean','max','min', 'count'])
# отфильтруем режиссеров, кто снял всего один или два фильма - нам нужны только многостаночники!
dir = dir.query('count > 2')

# отсортируем по убыванию медианы сборов, и вот наш список фавориов на прокат
dir.astype('int64').sort_values('median', ascending=False).nlargest(10, 'median')

,median,mean,max,min,count
director,,,,,
Pierre Coffin,970761885,890335610,1156730962,543513985,3
David Yates,946259303,1038573906,1327817822,933959197,4
Peter Jackson,940703594,811324210,1118888979,93525586,8
Lee Unkrich,864625978,830204715,1063171911,562816256,3
Conrad Vernon,746921274,682756634,919838758,381509870,3
Pete Docter,731342744,715955869,853708609,562816256,3
Alfonso CuarÃ³n,716392705,525385670,789804554,69959751,3
Marc Webb,705717432,506218674,752215857,60722734,3
Sam Raimi,637817444,515177725,890871626,12008642,6


Как и ожидалось, половина из списка - режессеры Анимации или Фэнтези. Ну и фантастики. Но в любом случае - это все семейный просмотр.

Мне кажется, я уже достаточно для первого взгляда на вопрос разобрал проблему выбора стратегии составления сетки проката и принципов отбора фильмов. 